In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import numpy as np
from collections import defaultdict


from PIL import Image, ImageFilter
import io
import re
import random
import numpy.random as npr
from skimage import data
from scipy.ndimage import rotate
from kernels import *
import torchvision
from torchvision import models
import os
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
 
import my_utils as ut
import old_utils_averaged_filters as old_ut_avg
from transformers import Swinv2ForImageClassification, SwinConfig
from torch.optim import AdamW
from torchvision import transforms, datasets



In [13]:

transform = transforms.Compose([
   
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
   
])

# Create dataset instances
train_dataset = old_ut_avg.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='train')
val_test_dataset = old_ut_avg.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='val')



# train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 3000, 3750, 3750)
train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 8, 8, 8)


train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)



In [14]:
#check distribution of classes 

print("Check overlap between datasets:\n")
print("\nTrain Subset and Validation Subset:")
ut.check_data_overlap(train_subset, val_subset)
print("\nTrain Subset and Test Subset:")
ut.check_data_overlap(train_subset, test_subset)
print("\nValidation Subset and Test Subset:")
ut.check_data_overlap(val_subset, test_subset)

print()
print("\nTrain Subset Distribution:\n")
ut.print_model_class_distribution(train_dataset, train_subset.indices)
print("\nValidation Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, val_subset.indices)
print("\nTest Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, test_subset.indices)

Check overlap between datasets:


Train Subset and Validation Subset:
No actual data overlap detected.

Train Subset and Test Subset:
No actual data overlap detected.

Validation Subset and Test Subset:
No actual data overlap detected.


Train Subset Distribution:

Total samples in subset: 24
Model ADM, Class ai: 2 (8.33%)
Model ADM, Class nature: 1 (4.17%)
Model BigGAN, Class nature: 3 (12.50%)
Model Midjourney, Class nature: 3 (12.50%)
Model VQDM, Class nature: 3 (12.50%)
Model glide, Class ai: 2 (8.33%)
Model glide, Class nature: 1 (4.17%)
Model stable_diffusion_v_1_4, Class ai: 2 (8.33%)
Model stable_diffusion_v_1_4, Class nature: 1 (4.17%)
Model stable_diffusion_v_1_5, Class nature: 2 (8.33%)
Model stable_diffusion_v_1_5, Class ai: 1 (4.17%)
Model wukong, Class ai: 2 (8.33%)
Model wukong, Class nature: 1 (4.17%)

Validation Subset Distribution:

Total samples in subset: 32
Model ADM, Class nature: 3 (9.38%)
Model ADM, Class ai: 1 (3.12%)
Model BigGAN, Class nature: 3 (9.38%)
Model

In [15]:
class SwinClassificationOldest(nn.Module):
    def __init__(self):
        super(SwinClassificationOldest, self).__init__()
        self.feature_combiner = old_ut_avg.CNNBlockOld()
        self.feature_combiner2 = old_ut_avg.CNNBlockOld()
        config = SwinConfig.from_pretrained('microsoft/swinv2-tiny-patch4-window8-256',num_classes=2)
        self.transformer = Swinv2ForImageClassification.from_pretrained(
            "microsoft/swinv2-tiny-patch4-window8-256",
            config=config
        )
        
        self.transformer.classifier = nn.Linear(config.hidden_size, 2) 

 
    def forward(self, rich, poor):
 
        
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor) 
        feature_difference = x - y
        outputs = self.transformer(feature_difference)

        return outputs.logits

In [16]:
class BaseClassifierOldest(nn.Module):
    def __init__(self):
        super(BaseClassifierOldest, self).__init__()
        self.feature_combiner = old_ut_avg.CNNBlockOld()
        self.feature_combiner2 = old_ut_avg.CNNBlockOld()
        self.classifier = ut.DeepClassifier() 

 
    def forward(self, rich, poor):
       
  
        
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor) 
        feature_difference = x - y
        outputs = self.classifier(feature_difference)

        return outputs

In [17]:
class ResNetClassifierOldest(nn.Module):
    def __init__(self):
        super(ResNetClassifierOldest, self).__init__()
        self.feature_combiner = old_ut_avg.CNNBlockOld()
        self.feature_combiner2 = old_ut_avg.CNNBlockOld()
        resnet_weights = models.ResNet50_Weights.DEFAULT
        self.resnet = models.resnet50(weights=resnet_weights)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])
        # Add a new classifier layer
        self.classifier = nn.Linear(self.resnet.fc.in_features, 2)
        # Adaptive pool to make sure output from feature maps is of fixed size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

 
    def forward(self, rich, poor):

        
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)    
        feature_difference = x - y
        
        # Process feature difference through the ResNet features
        features = self.features(feature_difference)
        pooled_features = self.adaptive_pool(features)
        flat_features = torch.flatten(pooled_features, 1)
        outputs = self.classifier(flat_features)

        return outputs

# Freeze the pretrained model oldest preprocessing and train 

In [18]:
#Swin Transformer
print("Training SWIN Frozen w/OldPre\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SwinClassificationOldest().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_OldestPre_SWIN_frozen.pth'

    #freeze the transformer layers
    for param in model.transformer.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.transformer.classifier.parameters():
        param.requires_grad = True

    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


No saved model found. Starting fresh!
Epoch 1/10
,Train Loss: 0.7794, Train Acc: 0.4167, Val Loss: 0.6121, Val Acc: 0.6562

Saved new best model with accuracy: 0.6562
Epoch 2/10
,Train Loss: 0.8181, Train Acc: 0.3750, Val Loss: 0.5870, Val Acc: 0.6875

Saved new best model with accuracy: 0.6875
Epoch 3/10
,Train Loss: 0.7028, Train Acc: 0.5417, Val Loss: 0.5949, Val Acc: 0.7188

Saved new best model with accuracy: 0.7188
Epoch 4/10
,Train Loss: 0.7491, Train Acc: 0.5000, Val Loss: 0.5945, Val Acc: 0.6875

Epoch 5/10
,Train Loss: 0.7808, Train Acc: 0.4167, Val Loss: 0.5910, Val Acc: 0.6562

Epoch 6/10
,Train Loss: 0.7582, Train Acc: 0.4583, Val Loss: 0.6012, Val Acc: 0.6562

Epoch 7/10
,Train Loss: 0.7288, Train Acc: 0.4583, Val Loss: 0.5973, Val Acc: 0.6562

Epoch 8/10
,Train Loss: 0.7179, Train Acc: 0.4583, Val Loss: 0.5997, Val Acc: 0.6562

Epoch 9/10
,Train Loss: 0.7673, Train Acc: 0.3750, Val Loss: 0.6143, Val Acc: 0.5938

Epoch 10/10
,Train Loss: 0.7281, Train Acc: 0.5000, Val Los

In [19]:
#ResNET

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training RES Frozen w/OldPre\n")
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ResNetClassifierOldest().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_OldestPre_RESNET_frozen.pth'

    #freeze resnet layers
    for param in model.resnet.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



No saved model found. Starting fresh!
Epoch 1/10
,Train Loss: 0.7059, Train Acc: 0.4583, Val Loss: 0.7654, Val Acc: 0.4688

Saved new best model with accuracy: 0.4688
Epoch 2/10
,Train Loss: 0.7052, Train Acc: 0.4167, Val Loss: 0.7391, Val Acc: 0.4688

Epoch 3/10
,Train Loss: 0.6811, Train Acc: 0.5833, Val Loss: 0.7284, Val Acc: 0.4688

Epoch 4/10
,Train Loss: 0.6935, Train Acc: 0.5833, Val Loss: 0.7109, Val Acc: 0.4688

Epoch 5/10
,Train Loss: 0.7043, Train Acc: 0.6250, Val Loss: 0.7211, Val Acc: 0.5000

Saved new best model with accuracy: 0.5000
Epoch 6/10
,Train Loss: 0.6829, Train Acc: 0.5833, Val Loss: 0.7157, Val Acc: 0.5000

Epoch 7/10
,Train Loss: 0.6678, Train Acc: 0.6250, Val Loss: 0.7170, Val Acc: 0.5000

Epoch 8/10
,Train Loss: 0.6786, Train Acc: 0.5833, Val Loss: 0.7106, Val Acc: 0.5000

Epoch 9/10
,Train Loss: 0.6795, Train Acc: 0.6250, Val Loss: 0.7008, Val Acc: 0.5000

Epoch 10/10
,Train Loss: 0.6659, Train Acc: 0.6250, Val Loss: 0.7011, Val Acc: 0.5000



## unfreeze the pretrained models oldest preprocessing

In [20]:
#ResNET

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training RES UnFrozen w/OldPre\n")
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ResNetClassifierOldest().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_OldestPre_RESNET_Unfrozen.pth'

    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



No saved model found. Starting fresh!
Epoch 1/10
,Train Loss: 0.6911, Train Acc: 0.5000, Val Loss: 0.7175, Val Acc: 0.5000

Saved new best model with accuracy: 0.5000
Epoch 2/10
,Train Loss: 0.6483, Train Acc: 0.7500, Val Loss: 0.7091, Val Acc: 0.4062

Epoch 3/10
,Train Loss: 0.6203, Train Acc: 0.7500, Val Loss: 0.7098, Val Acc: 0.4375

Epoch 4/10
,Train Loss: 0.6040, Train Acc: 0.7500, Val Loss: 0.6966, Val Acc: 0.4375

Epoch 5/10
,Train Loss: 0.5446, Train Acc: 0.8333, Val Loss: 0.6971, Val Acc: 0.4375

Epoch 6/10
,Train Loss: 0.5360, Train Acc: 0.8333, Val Loss: 0.7034, Val Acc: 0.5000

Epoch 7/10
,Train Loss: 0.5250, Train Acc: 0.8333, Val Loss: 0.6983, Val Acc: 0.5000

Epoch 8/10
,Train Loss: 0.4890, Train Acc: 0.8750, Val Loss: 0.6861, Val Acc: 0.5312

Saved new best model with accuracy: 0.5312
Epoch 9/10
,Train Loss: 0.4283, Train Acc: 0.8333, Val Loss: 0.6814, Val Acc: 0.5625

Saved new best model with accuracy: 0.5625
Epoch 10/10
,Train Loss: 0.4370, Train Acc: 0.8333, Val Los

In [21]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training SWIN UnFrozen w/OldPre\n")
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SwinClassificationOldest().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_OldestPre_SWIN_Unfrozen.pth'
    
    
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


No saved model found. Starting fresh!
Epoch 1/10
,Train Loss: 0.8337, Train Acc: 0.2917, Val Loss: 1.0206, Val Acc: 0.4688

Saved new best model with accuracy: 0.4688
Epoch 2/10
,Train Loss: 0.7795, Train Acc: 0.6250, Val Loss: 0.9471, Val Acc: 0.4688

Epoch 3/10
,Train Loss: 0.6932, Train Acc: 0.6250, Val Loss: 0.8082, Val Acc: 0.4688

Epoch 4/10
,Train Loss: 0.6498, Train Acc: 0.6250, Val Loss: 0.7423, Val Acc: 0.4688

Epoch 5/10
,Train Loss: 0.6165, Train Acc: 0.6250, Val Loss: 0.7212, Val Acc: 0.4062

Epoch 6/10
,Train Loss: 0.6548, Train Acc: 0.6667, Val Loss: 0.7161, Val Acc: 0.4688

Epoch 7/10
,Train Loss: 0.6123, Train Acc: 0.7500, Val Loss: 0.7281, Val Acc: 0.4688

Epoch 8/10
,Train Loss: 0.6139, Train Acc: 0.6667, Val Loss: 0.7441, Val Acc: 0.4688

Epoch 9/10
,Train Loss: 0.5712, Train Acc: 0.7917, Val Loss: 0.7512, Val Acc: 0.4688

Epoch 10/10
,Train Loss: 0.5152, Train Acc: 0.7917, Val Loss: 0.7979, Val Acc: 0.4688



In [22]:
#Original Model
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training Base unFrozen w/OldPre\n")
for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BaseClassifierOldest().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_OldestPre_Base.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



No saved model found. Starting fresh!
Epoch 1/10
,Train Loss: 0.6766, Train Acc: 0.6667, Val Loss: 0.6928, Val Acc: 0.5312

Saved new best model with accuracy: 0.5312
Epoch 2/10
,Train Loss: 0.6535, Train Acc: 0.6667, Val Loss: 0.6932, Val Acc: 0.4688

Epoch 3/10
,Train Loss: 0.6381, Train Acc: 0.6667, Val Loss: 0.6934, Val Acc: 0.4688

Epoch 4/10
,Train Loss: 0.6215, Train Acc: 0.7083, Val Loss: 0.6936, Val Acc: 0.4688

Epoch 5/10
,Train Loss: 0.6108, Train Acc: 0.7083, Val Loss: 0.6938, Val Acc: 0.4688

Epoch 6/10
,Train Loss: 0.6001, Train Acc: 0.7083, Val Loss: 0.6939, Val Acc: 0.4688

Epoch 7/10
,Train Loss: 0.5901, Train Acc: 0.7083, Val Loss: 0.6941, Val Acc: 0.4688

Epoch 8/10
,Train Loss: 0.5833, Train Acc: 0.6667, Val Loss: 0.6944, Val Acc: 0.4688

Epoch 9/10
,Train Loss: 0.5726, Train Acc: 0.7083, Val Loss: 0.6946, Val Acc: 0.4688

Epoch 10/10
,Train Loss: 0.5653, Train Acc: 0.7500, Val Loss: 0.6949, Val Acc: 0.4688

